# Convert netcdf data to GeoTiff and cropping the area of interests. 
# Extract pixel data to pandas data frame for selected locations.

In [ ]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import rasterio as rio
import numpy as np
import matplotlib.image as mpimg
import xarray as xr
import os
import pandas as pd
import datetime

In [ ]:
# read netcdf files.  Change year here
fpath = "/home/user/project/data/"
pattern = '*.nc'

eflist = []

for root, dirs, files in os.walk(fpath):
    for name in files:
        eflist.append(os.path.join(fpath,name))
        

In [ ]:
det = nc.Dataset(eflist[1])
print(det)

In [ ]:
#for val in det.variables.values():
 #   print(val)

In [ ]:
det['time']

In [ ]:
tm = eflist[0].split('/')
tt = str(tm[6]).split('_')
tf = str(tt[3]).split('.')
#td = str(tf[3]).split('.')
tf


In [ ]:
eflist[1]

In [ ]:
# array to gtiff function. Change year here
x = -180
y = -90
res = 0.05
from rasterio.transform import Affine, from_origin 
transform = from_origin(-180,90, 0.05, 0.05)
#transform = Affine.translation(x - (res / 2), y - (res / 2)) * Affine.scale(res, res)
time = 'time'
var = 'bleaching_alert_area'


fpath = '/home/user/project/tiff/'
fname = 'crw_'
extname = '.tiff'



def array_to_gtiff(flist):
    for i in flist:
        dst = nc.Dataset(i)
        ds_sst = dst[var]
        ds_ar = np.array(ds_sst)
        #ds_ar = np.flipud(ds_ar)
        #ds_ar = np.flip(ds_ar)
        #ds_ar = np.fliplr(ds_ar)
        
        # time in seconds
        tdm = int(dst['time'][0])
        tdt = tdm + 3.469e+8 + 259200
        dt = datetime.datetime.fromtimestamp(tdt).strftime('%d-%m-%Y')
            #date_time.append(dt)
        
        #print(dt)
        
        
        temp = str(i).split('/')
        temp = str(temp[6]).split('_')
        temp = str(temp[3]).split('.')
        temp = str(temp[0])

        
        tname = fpath + fname + temp + extname
        
        tif_test = rio.open(tname, 
            'w',
            driver = 'GTiff',            
            height=3600,
            width=7200,
            dtype='int8',
            count = 1,
            crs="EPSG:4326",
            transform=transform)
    
        tif_test.write(ds_ar)
        tif_test.close()
        dst.close()

In [ ]:
array_to_gtiff(eflist)


In [ ]:
# chage year here

from rasterio.plot import show
import fiona
import rasterio.mask 

fpath = "/home/user/project/tifs/"
pattern = '*.tiff'

gtflist = []

for root, dirs, files in os.walk(fpath):
    for name in files:
        gtflist.append(os.path.join(fpath,name))

                
with fiona.open("/home/user/project/vector_data/aoi_polygon.shp") as shapefile:                
    shapes = [feature["geometry"]for feature in shapefile]

#out_image, out_transform = rasterio.mask.mask(testrs, shapes, crop = True)


In [ ]:
# change year here
tpath = '/home/user/project/tifs_clipped/'


def gtif_crop(my_list):
    
    templist = []
    
    for i in my_list:
        
        tid = str(i)
        tid = tid.split('/')
        tid = str(tid[6])
        #print(tid)
        
                
        tfname = tpath + tid
        
        src0 =  rasterio.open(i,'r') 
        meta = src0.meta
            
        out_image,out_transform = rasterio.mask.mask(src0, shapes, crop = True)
        
        out_meta = src0.meta
        
        out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
        
        clipped = rio.open(tfname,
                  'w',
                   **out_meta)
        
        clipped.write(out_image)
        clipped.close()
        
    

In [ ]:
gtif_crop(gtflist)

In [ ]:
tf = clp_flist[1]
tf1 = str(tf).split('/')
tf2 = str(tf1[8]).split('.')
tf3 = str(tf2[0]).split('_')[1]
tf3
#

# Extracting pixel values 

In [ ]:
# change year here

import rasterio
import geopandas as gpd
import os

# make list of rasters 
clpath = '/home/user/project/tifs_clipped/'
pattern = '*.tiff'

clp_flist = []

for root, dirs, files in os.walk(clpath):
    for name in files:
        clp_flist.append(os.path.join(clpath,name))

# Read points from shapefile
pts = gpd.read_file('/home/user/project/vector_Data/monitoring_stations.shp')

pts = pts[['Name', 'LON', 'LAT', 'geometry']]
pts.index = range(len(pts))
coords = [(x,y) for x, y in zip(pts.LON, pts.LAT)]

date_time = []

for i in clp_flist:
    mid = str(i).split('/')
    mid = str(mid[8]).split('.')    
    mid = str(mid[0]).split('_')[1]
    
    t_yr = mid[0:4]
    t_mn = mid[4:6]
    t_dy = mid[6:]
    
    t_dt_ = t_dy + '-' + t_mn + '-' + t_yr
    date_time.append(t_dt_)
    
    # Open the raster and store metadata

    src = rio.open(i) 
    
    
    # Sample the raster at every point location and store values in DataFrame
    pts[mid] = [x for x in src.sample(coords)]
    pts[mid] = pts.apply(lambda x: x[mid][0], axis=1)
    
    src.close() 

In [ ]:
pts = pts.T
pts

In [ ]:
#tlist = [0,0,0,0]
#for i in range(len(tlist)):
#    date_time.insert(i, tlist[i])
#print(date_time)

pts = pts.drop(pts.index[[1,2,3]])
pts.columns = ['Clevelend Bay ','Agincourt Reef', 'Masig Reef', 'Square Rocks', 'Hardy Reef', 'Davies Reef', 'Heron Island', 'Lizard Island', 'Thursday']
print(pts)

In [ ]:
date_time.insert(0,0)
pts['Date'] = date_time
pts

In [ ]:
# change year here
pts.to_csv('/home/user/project/extrated/baa_2011.csv')

In [ ]:
pts.shape